In [1]:
from inspect import Signature, Parameter

# Define parameters for a function signature (x, y=42, *, z=None)
parms = [
    Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
    Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
    Parameter('z', Parameter.KEYWORD_ONLY, default=None)
]

# Create the signature object
sig = Signature(parms)

# Print the function signature
print(sig)  # Output: (x, y=42, *, z=None)


(x, y=42, *, z=None)


In [4]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)  # Bind the provided args to the signature
    for name, value in bound_values.arguments.items():
        print(name, value)

# Test cases
func(1, 2, z=3)  
# Output:
# x 1
# y 2
# z 3

func(1)  
# Output:
# x 1

func(1, z=3)  
# Output:
# x 1
# z 3

func(y=2, x=1)  
# Output:
# x 1
# y 2

try:
    func(1, 2, 3, 4)  # Raises TypeError: too many positional arguments
except TypeError as e:
    print("too many positional arguments")

try:
    func(y=2)         # Raises TypeError: 'x' parameter lacking default value
except TypeError as e:
    print("'x' parameter lacking default value")

try:
    func(1, y=2, x=3) # Raises TypeError: multiple values for argument 'x'
except TypeError as e:
    print("multiple values for argument 'x'")


x 1
y 2
z 3
x 1
x 1
z 3
x 1
y 2
too many positional arguments
'x' parameter lacking default value
multiple values for argument 'x'


In [6]:
from inspect import Signature, Parameter

# Helper function to create a signature from field names
def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()  # Default empty signature

    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)  # Enforce signature
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)  # Assign values to attributes

# Define subclasses with specific argument signatures
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')

# Testing
import inspect
print(inspect.signature(Stock))  # Output: (name, shares, price)

s1 = Stock('ACME', 100, 490.1)  # Works fine
try:
    s2 = Stock('ACME', 100)  # Raises TypeError: 'price' parameter lacking default value
    s3 = Stock('ACME', 100, 490.1, shares=50)  # Raises TypeError: multiple values for 'shares'
except TypeError as e:
    print(e)


(name, shares, price)
missing a required argument: 'price'


In [9]:
from inspect import Signature, Parameter

# Helper function to create a signature from field names
def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

# Metaclass to enforce signatures
class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields', []))  
        return super().__new__(cls, clsname, bases, clsdict)

# Base class using the metaclass
class Structure(metaclass=StructureMeta):
    _fields = []  # Default empty fields

    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Define subclasses with fields
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']

# Testing
import inspect
print(inspect.signature(Stock))  # Output: (name, shares, price)
print(inspect.signature(Point))  # Output: (x, y)

s1 = Stock('ACME', 100, 490.1)  # Works fine
try:
    s2 = Stock('ACME', 100)  # Raises TypeError: 'price' parameter lacking default value
    s3 = Stock('ACME', 100, 490.1, shares=50)  # Raises TypeError: multiple values for 'shares'
except TypeError as e:
    print(e)


(name, shares, price)
(x, y)
missing a required argument: 'price'
